In [1]:
pip install transformers sentence-transformers datasets trl torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 require

In [2]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=c1c342e6add0f862189f1c8578588029a52991ec6fe2fdb1ab100a2aaf078ac7
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers import SentenceTransformer
from googletrans import Translator
from dask import delayed, compute
import torch
import numpy as np
from trl import PPOTrainer, PPOConfig
from datasets import Dataset
import cloudpickle as pickle


# Initialize models and tokenizers
t5_model = T5ForConditionalGeneration.from_pretrained('Vamsi/T5_Paraphrase_Paws')
t5_tokenizer = T5Tokenizer.from_pretrained('Vamsi/T5_Paraphrase_Paws')

nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
nli_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

similarity_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Translator for back-translation
# translator = Translator()

# Sample text
text = """Before the formation of kamikaze units, pilots had made deliberate crashes as a last resort when their aircraft
          had suffered severe damage and they did not want to risk being captured or wanted to do as much damage to
          the enemy as possible, since they were crashing anyway. Such situations occurred in both the Axis and Allied
          air forces. Axell and Kase see these suicides as 'individual, impromptu decisions by men who were mentally
          prepared to die'."""

# Function to perform paraphrasing using T5
def paraphrase_t5(text):
    input_text = f"paraphrase: {text} </s>"
    input_ids = t5_tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = t5_model.generate(input_ids, max_length=512, num_beams=4, num_return_sequences=2, early_stopping=True)
    paraphrases = [t5_tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in outputs]
    return paraphrases

# Function for back-translation (English -> another language -> English)
# def back_translation(text):
#     translated = translator.translate(text, src='en', dest='fr').text  # Translate to French
#     back_translated = translator.translate(translated, src='fr', dest='en').text  # Back to English
#     return back_translated

# Function for NLI-based validation
def validate_nli(original_text, paraphrased_text):
    inputs = nli_tokenizer.encode_plus(original_text, paraphrased_text, return_tensors="pt", truncation=True)
    logits = nli_model(**inputs).logits
    probabilities = torch.softmax(logits, dim=-1).detach().cpu().numpy()[0]
    entailment_prob = probabilities[2]  # Index 2 is for "entailment"
    return entailment_prob > 0.85  # Entailment threshold

# Define a reward function for RL
def reward_function(paraphrase, original):
    # Calculate semantic similarity (higher similarity gets a higher reward)
    original_embedding = similarity_model.encode([original])[0]
    paraphrase_embedding = similarity_model.encode([paraphrase])[0]
    similarity_score = np.dot(original_embedding, paraphrase_embedding) / (np.linalg.norm(original_embedding) * np.linalg.norm(paraphrase_embedding))

    # Use NLI validation as a part of the reward
    entailment_valid = validate_nli(original, paraphrase)

    # Reward based on both similarity and entailment (reward between 0-1)
    return similarity_score * entailment_valid

# Reinforcement Learning using PPO (Proximal Policy Optimization)
def train_paraphraser_with_rl(paraphrases, original_text):
    # RL config
    config = PPOConfig()
    trainer = PPOTrainer(t5_model, config)

    # Prepare dataset
    dataset = Dataset.from_dict({"text": paraphrases})

    for epoch in range(1):  # One epoch for demonstration; increase as needed
        for sample in dataset:
            paraphrase = sample["text"]
            reward = reward_function(paraphrase, original_text)
            trainer.step(paraphrase, reward)  # Update model with reward

    # Return optimized paraphrases
    return paraphrases

# Post-Processing: Style Matching
def style_matching(text, target_style='formal'):
    style_model = T5ForConditionalGeneration.from_pretrained('mrm8488/t5-base-finetuned-sentence-style-transfer')
    style_tokenizer = T5Tokenizer.from_pretrained('mrm8488/t5-base-finetuned-sentence-style-transfer')

    input_text = f"transfer {target_style}: {text} </s>"
    input_ids = style_tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = style_model.generate(input_ids, max_length=512, num_beams=4, num_return_sequences=1, early_stopping=True)
    styled_text = style_tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return styled_text

# Main function to process text
@delayed
def process_text(text):
    # Step 1: Paraphrase using T5
    paraphrases = paraphrase_t5(text)

    # Step 2: Apply back-translation
    paraphrases = [p for p in paraphrases]

    # Step 3: Apply Reinforcement Learning to optimize paraphrasing
    paraphrases = train_paraphraser_with_rl(paraphrases, text)

    # Step 4: Validate using NLI for semantic preservation
    valid_paraphrases = [p for p in paraphrases if validate_nli(text, p)]

    # Step 5: Style matching for formal tone (optional)
    styled_paraphrases = [style_matching(p) for p in valid_paraphrases]

    # If no paraphrases pass validation, return the original text
    if not styled_paraphrases:
        return text

    return styled_paraphrases[0]  # Return the first valid and styled paraphrase

# Dask for scalability - parallel processing
texts = [text]  # Replace with a list of texts if processing multiple
results = compute(*[process_text(t) for t in texts], scheduler='processes')

# Output the transformed content
for result in results:
    print("Transformed Content:", result)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]